In [40]:
import ghmm
import pickle
import numpy as np
from itertools import product
from pprint import pprint
import pysam
import os
import pandas
import re
import editdistance
import sys
from multiprocessing import Pool
%run "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/alignment_lib.ipynb"

In [41]:
NMERS = 5
NSTATES = 4**NMERS

In [42]:
# args = {
#     "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_events.template.pickle",
#     "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_calling",
#     "ref": "/home/ibis/gregor.sturm/nanopore/david_eccles_bc_ideas/mouse_ref.fa",
#     "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/david_hmm_params.pickle",
#     "ncores": 4
# }

args = {
    "events" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_events.template.pickle",
    "out_basename" : "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_calling",
    "ref": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_ref.fasta",
    "hmm_params": "/home/ibis/gregor.sturm/nanopore/own/notebooks/03_pipeline/lambda_hmm_params.pickle",
    "ncores": 62
}

In [43]:
HMM_PARAMS = pickle.load(open(args["hmm_params"], 'rb'))
HMM_PARAMS = sorted(zip(*[HMM_PARAMS["kmers"], HMM_PARAMS["means"], HMM_PARAMS["stdv"]]))

# Train Model 

In [44]:
def mk_transmat(nmers):
    """make a transition matrix assuming move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 1/4. if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)] else 0.
            transmat[j, i] = p          
            
    return transmat.tolist()

In [45]:
def mk_transmat0(nmers):
    """make a transition matrix assuming move=0 or move=1"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (9/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (1/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [46]:
def mk_transmat2(nmers):
    """make a transition matrix assuming move=0 or move=1 or move=2"""
    all_kmers = [x[0] for x in HMM_PARAMS]
    n_components = len(all_kmers)
    transmat = np.empty((n_components, n_components))
    for j, from_kmer in enumerate(all_kmers):
        for i, to_kmer in enumerate(all_kmers):
            p = 0
            if from_kmer[-(NMERS-2):] == to_kmer[:(NMERS-2)]:
                """move=2"""
                p = (3/10.) * (1/16.)
            elif from_kmer[-(NMERS-1):] == to_kmer[:(NMERS-1)]:
                """move=1"""
                p = (7/10.) * (1/4.) 
            elif from_kmer == to_kmer:
                """move=0"""
                p = (0/10.) * 1
            transmat[j, i] = p          
            
    return transmat.tolist()

In [47]:
# example code for a continuous HMM with gaussian emissions


F = ghmm.Float()  # emission domain of this model

A = mk_transmat2(NMERS)
B = [[x[1], x[2]] for x in HMM_PARAMS]   # parameters of emission distributions in pairs of (mu, sigma)
pi = [1/float(NSTATES)] * NSTATES   # initial probabilities per state

# generate model from parameters
model = ghmm.HMMFromMatrices(F,ghmm.GaussianDistribution(F), A, B, pi)

In [48]:
s = str(model)
print(s)

GaussianEmissionHMM(N=64)
  state 0 (initial=0.00, mu=67.75, sigma=3.49)
    Transitions: ->0 (0.02)
  state 1 (initial=0.00, mu=61.09, sigma=3.02)
    Transitions: ->1 (0.02)

  ...

  state 62 (initial=0.00, mu=52.22, sigma=2.39)
    Transitions: ->62 (0.02)
  state 63 (initial=0.00, mu=47.64, sigma=2.26)
    Transitions: ->63 (0.02)



In [49]:
def result_to_seq(result):
    states = result[0]
    all_kmers = [x[0] for x in HMM_PARAMS]
    kmers = [all_kmers[x] for x in states]
    seq = [kmer[NMERS/2] for kmer in kmers]
    return "".join(seq)

In [50]:
def predict(means):
    seq = ghmm.EmissionSequence(F, means)
    result = model.viterbi(seq)
    return result_to_seq(result)

In [51]:
predict([44.2, 44.3, 56, 58.2, 56.2, 58.1, 58.2, 60, 30.2])

u'TTTTTTTTT'

# Validate Model 

In [52]:
!pwd

/home/ibis/gregor.sturm/nanopore/own/notebooks


In [53]:
assert os.path.isfile(args["events"])

In [54]:
ref_file = args["ref"]
test = !cat {ref_file} | grep ">"
print(test)
ref = !cat {ref_file} | grep -v ">"
ref = ref[0]
print(ref[:100])

['>burn-in lambda_ref']
GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC


In [55]:
file_data = pickle.load(open(args["events"], 'rb'))
file_data = [f for f in file_data if f is not None]

In [56]:
prepare_filemap(file_data)

### Make Alignment

In [57]:
fastq_file = "{0}.fastq".format(args["out_basename"])
mk_fastq(fastq_file, file_data)

In [58]:
sam_file = "{0}.sam".format(args["out_basename"])
graphmap(ref_file, fastq_file, sam_file, args["ncores"])

[Index 17:26:03] Running in fast and sensitive mode. Two indexes will be used (double memory consumption).
[Index 17:26:03] Index already exists. Loading from file.
[Index 17:26:03] Secondary index already exists. Loading from file.
[Index 17:26:04] Index loaded in 0.46 sec.
[Index 17:26:04] Memory consumption: [currentRSS = 516 MB, peakRSS = 21837 MB]

[Run 17:26:04] Automatically setting the maximum allowed number of regions: max. 500, attempt to reduce after 100
[Run 17:26:04] Reference genome is assumed to be linear.
[Run 17:26:04] Only one alignment will be reported per mapped read.
[ProcessReads 17:26:04] Reads will be loaded in batches of up to 200 MB in size.
[ProcessReads 17:26:04] Batch of 6123 reads (64 MiB) loaded in 0.47 sec. (37348520 bases)
[ProcessReads 17:26:04] Memory consumption: [currentRSS = 582 MB, peakRSS = 21837 MB]
[ProcessReads 17:26:04] Using 62 threads.
[ProcessReads 17:26:59] [CPU time: 1942.49 sec, RSS: 647 MB] Read: 6123/6123 (100.00%) [m: 5709, u: 414]  

In [59]:
bam_file = prepare_sam(args["out_basename"])

In [60]:
samfile = pysam.AlignmentFile(bam_file)
reads  = [x for x in samfile.fetch()]
len(reads)

5709

In [ ]:
def basecall_read(params):
    read_name, ref_pos = params
    file_id, channel_id = get_file_and_channel(read_name)
    print(file_id, channel_id, read_name)
    sys.stdout.flush()
    
    file_obj = get_file(channel_id, file_id)
      
    events = [x["mean"] for x in file_obj["events"].to_dict("records")]
    metrichor_seq = file_obj["fastq"].split("\n")[1]
    ref_seq = "".join([ref[x] for x in ref_pos])
    called_seq = predict(events)
    
    stats = {
        "d_metrichor": int(editdistance.eval(ref_seq, metrichor_seq)),
        "d_caller": int(editdistance.eval(ref_seq, called_seq)), 
        "length": len(ref_seq),
    }

    return (read_name, called_seq, stats)

In [ ]:
p = Pool(args["ncores"])
## one can not access a read in parallel (deadlock for whatever reason)
## therfore prepare input parameters outside of map
input_params = [(read.query_name, read.get_reference_positions()) for read in reads]
try:
    results = p.map(basecall_read, input_params)
except KeyboardInterrupt:
    p.terminate()

### Stats

In [ ]:
headers, seqs, stats = zip(*results)

In [ ]:
stats = pandas.DataFrame(list(stats))

In [ ]:
stats = stats.sum(0)

In [ ]:
stats

In [ ]:
print("Relative Performance: {0:5.3f}%".format(stats["d_metrichor"] * 100 /float(stats["d_caller"])))

In [ ]:
fasta_file_called = "{0}.called.fa".format(args["out_basename"])
with open(fasta_file_called, 'w') as f: 
    for header, seq in zip(headers, seqs): 
        f.write(">" + header + "\n")
        f.write(seq + "\n")

In [ ]:
sam_file_called = "{0}.called.sam".format(args["out_basename"])
graphmap(ref_file, fasta_file_called, sam_file_called, args["ncores"])

In [ ]:
prepare_sam("{0}.called".format(args["out_basename"]))